In [60]:
import pandas as pd
import matplotlib.pyplot as plt

In [ ]:

df = pd.read_json("data.json")

### DATA EXPLORATION

In [47]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 521 entries, 0 to 520
Data columns (total 14 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   card_length       439 non-null    float64
 1   card_length_unit  439 non-null    object 
 2   RAM               498 non-null    float64
 3   RAM_unit          498 non-null    object 
 4   chipset           486 non-null    object 
 5   clock_speed       407 non-null    float64
 6   clock_speed_unit  407 non-null    object 
 7   merging           490 non-null    object 
 8   rating_count      521 non-null    int64  
 9   questions_count   521 non-null    int64  
 10  purchases_count   521 non-null    int64  
 11  rating            237 non-null    float64
 12  name              521 non-null    object 
 13  price             513 non-null    float64
dtypes: float64(5), int64(3), object(6)
memory usage: 57.1+ KB


In [48]:
for col in df.columns:
    not_null_count = df[col].isnull().sum()
    num_rows = df.shape[0]
    print(f"{col:18} : ({(not_null_count / num_rows) * 100:5.2f}%) null columns")

card_length        : (15.74%) null columns
card_length_unit   : (15.74%) null columns
RAM                : ( 4.41%) null columns
RAM_unit           : ( 4.41%) null columns
chipset            : ( 6.72%) null columns
clock_speed        : (21.88%) null columns
clock_speed_unit   : (21.88%) null columns
merging            : ( 5.95%) null columns
rating_count       : ( 0.00%) null columns
questions_count    : ( 0.00%) null columns
purchases_count    : ( 0.00%) null columns
rating             : (54.51%) null columns
name               : ( 0.00%) null columns
price              : ( 1.54%) null columns


In [49]:
for col in df.columns:
    not_null_count = df[col].isnull().sum()
    num_rows = df.shape[0]
    print(f"{col:18} : ({(not_null_count / num_rows) * 100:5.2f}%) null columns")

card_length        : (15.74%) null columns
card_length_unit   : (15.74%) null columns
RAM                : ( 4.41%) null columns
RAM_unit           : ( 4.41%) null columns
chipset            : ( 6.72%) null columns
clock_speed        : (21.88%) null columns
clock_speed_unit   : (21.88%) null columns
merging            : ( 5.95%) null columns
rating_count       : ( 0.00%) null columns
questions_count    : ( 0.00%) null columns
purchases_count    : ( 0.00%) null columns
rating             : (54.51%) null columns
name               : ( 0.00%) null columns
price              : ( 1.54%) null columns


In [50]:
for col in df.select_dtypes(include=['object']).columns:
    num_uniques = df[col].nunique()
    if num_uniques < 10:
        print(f"{col:18} : {df[col].unique()} ")
        
print(df[df['RAM_unit']=='MB'].shape[0])

card_length_unit   : ['mm' None] 
RAM_unit           : ['GB' 'MB' None] 
clock_speed_unit   : ['MHz' None] 
merging            : ['Nie' 'NVLink' 'CrossFire' None 'SLI'] 
7


<p> 
It is evident that certain GPUs use MB to specify the RAM capacity as a measurement unit.
Therefore I will transform these outliers to represent RAM memory in GB.
</p>

In [51]:
df['RAM_GB'] = df['RAM'].copy()
df.loc[df['RAM_unit'] == 'MB', 'RAM_GB'] = df.apply(
    lambda x: x['RAM'] / 1024, axis=1
)
df.loc[df['merging'] == 'Nie', 'merging'] = 'No'

In [52]:
# card length unit only in mm; clock speed unit only in MHz
df['card_length_mm'] = df['card_length'].copy()
df['clock_speed_MHz'] = df['clock_speed'].copy()

cols_to_drop = ['RAM_unit','RAM','clock_speed_unit','card_length_unit','card_length','clock_speed']

df.drop(columns=cols_to_drop, inplace=True)
df['RAM_GB'].value_counts().sort_index(ascending=True)

RAM_GB
0.5       7
1.0      15
2.0      29
4.0      55
6.0      31
8.0     143
10.0      4
12.0     54
16.0    104
20.0     14
24.0     14
32.0     16
48.0     10
64.0      1
94.0      1
Name: count, dtype: int64

In [59]:
pd.cut(df['clock_speed_MHz'],bins=10).value_counts().sort_index(ascending=True)

clock_speed_MHz
(1069.011, 1269.9]     17
(1269.9, 1468.8]       40
(1468.8, 1667.7]       32
(1667.7, 1866.6]       65
(1866.6, 2065.5]        5
(2065.5, 2264.4]       15
(2264.4, 2463.3]       36
(2463.3, 2662.2]      144
(2662.2, 2861.1]       38
(2861.1, 3060.0]        8
Name: count, dtype: int64

In [ ]:
print(f"Missing values in price: {df['price'].isna().sum()}")
print(f"Dataframe size before dropping rows: {df.shape[0]}")
df[df['price'].isna()==True]

# It will not make any sense to keep the products with missinng prices, so I'm going to drop them
df = df[df['price'].isna()==False]
print(f"Dataframe size after dropping rows witth missing price: {df.shape[0]}")

Missing values in price: 8
Dataframe size before dropping rows: 521
Dataframe size after dropping rows witth missing price: 513


In [55]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 513 entries, 0 to 520
Data columns (total 11 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   chipset          478 non-null    object 
 1   merging          482 non-null    object 
 2   rating_count     513 non-null    int64  
 3   questions_count  513 non-null    int64  
 4   purchases_count  513 non-null    int64  
 5   rating           231 non-null    float64
 6   name             513 non-null    object 
 7   price            513 non-null    float64
 8   RAM_GB           490 non-null    float64
 9   card_length_mm   432 non-null    float64
 10  clock_speed_MHz  400 non-null    float64
dtypes: float64(5), int64(3), object(3)
memory usage: 48.1+ KB


### New columns

In [ ]:
df['PricePer1GB'] = round(df['price'] / df['RAM_GB'])
df['PricePer100MHz'] = round(100 * df['price'] / df['clock_speed_MHz'])
df[['PricePer1GB', 'PricePer100MHz']].describe()

,PricePer1GB,PricePer100MHz
count,490.000000,400.000000
mean,283.522331,209.144713
std,353.000639,506.676974
min,51.675000,23.459387
25%,151.567187,56.904549
50%,211.725625,87.829314
75%,301.445000,180.417785
max,5507.800000,8463.966794


In [64]:
df.columns

Index(['chipset', 'merging', 'rating_count', 'questions_count',
       'purchases_count', 'rating', 'name', 'price', 'RAM_GB',
       'card_length_mm', 'clock_speed_MHz', 'PricePerGB', 'PricePer1GB',
       'PricePer100MHz'],
      dtype='object')